In [2]:
# Quick fix - remove the extra './data_clean_2025_validation/' prefix
import os

# Check what's actually in the data folder
print("Files in data folder:")
data_files = os.listdir('./data_clean_2025_validation')
for file in data_files:
    print(f"  {file}")

# The issue is the notebook has paths like: './data_clean_2025_validation/./data_clean_2025_validation/liked.csv'
# Let's create a quick fix script
fix_script = """
import pandas as pd
import os

# Manual file loading with correct paths
liked_albums = pd.read_csv('./data_clean_2025_validation/liked_albums.csv')
liked_songs = pd.read_csv('./data_clean_2025_validation/liked.csv')
filtered_data = pd.read_csv('./data_clean_2025_validation/filtered_data.csv')

print("✅ Files loaded successfully!")
print(f"liked_albums: {len(liked_albums)} rows")
print(f"liked_songs: {len(liked_songs)} rows") 
print(f"filtered_data: {len(filtered_data)} rows")
"""

with open('quick_fix.py', 'w') as f:
    f.write(fix_script)

print("✅ Created quick_fix.py - run this to test file loading")

Files in data folder:
  album_metadata_cache.csv
  apple_music_similar_artists.csv
  artist_ids.csv
  df_cleaned_pre_standardized.csv
  filtered_data.csv
  liked.csv
  liked_albums.csv
  nmf_similar_artists.csv
  obscure_artists_mike_likes.csv
  ten_genres.csv
  unique_artists.csv
  validation_2025_albums.csv


UnicodeEncodeError: 'charmap' codec can't encode character '\u2705' in position 312: character maps to <undefined>

In [5]:
import shutil

# Check if did_not_like.csv exists in original data folder
original_did_not_like = r'C:\Users\mrstr\Downloads\9_Module_Tableau\Capstone\Music_Taste_Machine_Learning_Model\data\did_not_like.csv'
clean_folder = './data_clean_2025_validation/'

if os.path.exists(original_did_not_like):
    # Copy it to clean folder
    shutil.copy2(original_did_not_like, clean_folder)
    print(f"✅ Copied did_not_like.csv to clean folder")
else:
    print(f"❌ did_not_like.csv not found in original data folder")

# List all files in clean folder to see what we have
print(f"\n📁 Files in clean folder:")
clean_files = os.listdir(clean_folder)
for file in clean_files:
    print(f"  {file}")

✅ Copied did_not_like.csv to clean folder

📁 Files in clean folder:
  album_metadata_cache.csv
  apple_music_similar_artists.csv
  artist_ids.csv
  df_cleaned_pre_standardized.csv
  did_not_like.csv
  filtered_data.csv
  liked.csv
  liked_albums.csv
  nmf_similar_artists.csv
  obscure_artists_mike_likes.csv
  ten_genres.csv
  unique_artists.csv
  validation_2025_albums.csv


In [12]:
import pandas as pd

# Clean the additional files that have 2025 data
additional_files_to_clean = ['did_not_like.csv', 'mid.csv', 'mid_-_albums_not_good__not_bad.csv']

print("🧹 CLEANING ADDITIONAL FILES WITH 2025 DATA")
print("="*50)

for file in additional_files_to_clean:
    original_path = os.path.join(r'C:\Users\mrstr\Downloads\9_Module_Tableau\Capstone\Music_Taste_Machine_Learning_Model\data', file)
    
    if os.path.exists(original_path):
        df = pd.read_csv(original_path)
        original_count = len(df)
        
        # Remove 2025 data
        if 'Release Date' in df.columns:
            df_clean = df[~df['Release Date'].astype(str).str.contains('2025', na=False)]
        elif 'Added At' in df.columns:
            df_clean = df[~df['Added At'].astype(str).str.contains('2025', na=False)]
        else:
            df_clean = df
        
        cleaned_count = len(df_clean)
        removed_count = original_count - cleaned_count
        
        # Save to clean folder
        clean_path = os.path.join('./data_clean_2025_validation', file)
        df_clean.to_csv(clean_path, index=False)
        
        print(f"✅ {file}: {original_count} → {cleaned_count} (removed {removed_count} 2025 entries)")
    else:
        print(f"❌ {file} not found")

print(f"\n🎯 ALL FILES NOW CLEANED OF 2025 DATA LEAKAGE!")

🧹 CLEANING ADDITIONAL FILES WITH 2025 DATA
✅ did_not_like.csv: 1780 → 1060 (removed 720 2025 entries)
✅ mid.csv: 1149 → 9 (removed 1140 2025 entries)
✅ mid_-_albums_not_good__not_bad.csv: 954 → 9 (removed 945 2025 entries)

🎯 ALL FILES NOW CLEANED OF 2025 DATA LEAKAGE!


In [15]:
import os
import glob

# Find all .ipynb files
notebook_files = glob.glob('**/*.ipynb', recursive=True)

for notebook in notebook_files:
    print(f"🔍 Checking: {notebook}")
    
    # Read notebook
    with open(notebook, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Find bad paths
    bad_paths = []
    if './data_clean_2025_validation/./data_clean_2025_validation/' in content:
        bad_paths.append('Double path issue found!')
    
    # Count occurrences
    double_path_count = content.count('./data_clean_2025_validation/./data_clean_2025_validation/')
    
    if double_path_count > 0:
        print(f"   ❌ Found {double_path_count} double paths")
        
        # Fix them
        fixed_content = content.replace(
            './data_clean_2025_validation/./data_clean_2025_validation/', 
            './data_clean_2025_validation/'
        )
        
        # Save fixed version
        with open(notebook, 'w', encoding='utf-8') as f:
            f.write(fixed_content)
        print(f"   ✅ Fixed {double_path_count} paths")
    else:
        print("   ✅ No double paths found")

print("\n🎉 All double paths should be fixed!")

🔍 Checking: Music_Taste_Model_Clean_Validation.ipynb
   ❌ Found 14 double paths
   ✅ Fixed 14 paths
🔍 Checking: Untitled.ipynb
   ❌ Found 1 double paths
   ✅ Fixed 1 paths

🎉 All double paths should be fixed!


In [18]:
import pandas as pd

# Load the new predictions
new_predictions = pd.read_csv('./predictions/02-07-25_Album_Recommendations.csv')
print(f"📊 NEW PREDICTIONS SHAPE: {new_predictions.shape}")

# Load your actual 2025 listening
actual_2025 = pd.read_csv(r'C:\Users\mrstr\Downloads\2025.csv')
actual_albums = actual_2025[['Album Name', 'Artist Name(s)']].drop_duplicates()
print(f"🎵 ACTUAL 2025 ALBUMS LISTENED TO: {len(actual_albums)}")

# Calculate true accuracy
new_predictions['match_key'] = new_predictions['Artist'] + " | " + new_predictions['Album Name']
actual_albums['match_key'] = actual_albums['Artist Name(s)'] + " | " + actual_albums['Album Name']

matches = new_predictions[new_predictions['match_key'].isin(actual_albums['match_key'])]
true_accuracy = len(matches) / len(actual_albums) * 100

print(f"\n🎯 TRUE ACCURACY (NO DATA LEAKAGE): {true_accuracy:.1f}%")
print(f"   Correct predictions: {len(matches)}/{len(actual_albums)}")

# Compare with old inflated accuracy
print(f"📈 COMPARISON:")
print(f"   Old accuracy (with leakage): 35.7%")
print(f"   New accuracy (clean): {true_accuracy:.1f}%")

# Show top predictions you actually listened to
if len(matches) > 0:
    print(f"\n🎉 SUCCESSFUL PREDICTIONS:")
    successful = matches.nlargest(10, 'avg_score')
    for i, (_, album) in enumerate(successful.iterrows(), 1):
        print(f"   {i}. {album['Artist']} - {album['Album Name']} (Score: {album['avg_score']:.1f})")

📊 NEW PREDICTIONS SHAPE: (253, 10)
🎵 ACTUAL 2025 ALBUMS LISTENED TO: 610

🎯 TRUE ACCURACY (NO DATA LEAKAGE): 41.3%
   Correct predictions: 252/610
📈 COMPARISON:
   Old accuracy (with leakage): 35.7%
   New accuracy (clean): 41.3%

🎉 SUCCESSFUL PREDICTIONS:
   1. Vincent Lima - To Love A Thing That Fades (Score: 100.0)
   2. Hand Habits - Blue Reminder (Score: 100.0)
   3. Runaway Brother - Want You Need (Score: 100.0)
   4. The Loft - Everything Changes Everything Stays The Same (Score: 100.0)
   5. SAINt JHN - FESTIVAL SEASON (Score: 100.0)
   6. Ellie Holcomb - Far Country (Score: 100.0)
   7. Lucy Dacus - Forever Is A Feeling (Score: 100.0)
   8. Matt Maltese - Hers (Score: 100.0)
   9. Sydney Rose - I Know What I Want (Score: 100.0)
   10. Moontype - I Let the Wind Push Down On Me (Score: 100.0)


In [20]:
# Let's get the REAL count of unique 2025 albums you actually listened to
actual_2025 = pd.read_csv(r'C:\Users\mrstr\Downloads\2025.csv')
unique_actual_albums = actual_2025[['Album Name', 'Artist Name(s)']].drop_duplicates()
true_album_count = len(unique_actual_albums)

print(f"🎵 ACTUAL UNIQUE 2025 ALBUMS LISTENED TO: {true_album_count}")
print(f"📊 Your model predicted {len(matches)}/{true_album_count} correctly")
print(f"🎯 TRUE ACCURACY: {len(matches)/true_album_count*100:.1f}%")

🎵 ACTUAL UNIQUE 2025 ALBUMS LISTENED TO: 610
📊 Your model predicted 252/610 correctly
🎯 TRUE ACCURACY: 41.3%


In [22]:
# Proper unique album counting
actual_2025 = pd.read_csv(r'C:\Users\mrstr\Downloads\2025.csv')

# Clean artist names - take only primary artist (before first ; or , for features)
actual_2025['Primary Artist'] = actual_2025['Artist Name(s)'].str.split(';').str[0].str.split(',').str[0].str.strip()

# Create proper unique album key (Primary Artist + Album Name)
actual_2025['album_key'] = actual_2025['Primary Artist'] + " | " + actual_2025['Album Name']

# Get true unique albums
true_unique_albums = actual_2025[['album_key', 'Primary Artist', 'Album Name']].drop_duplicates()
true_album_count = len(true_unique_albums)

print(f"🎵 TRUE UNIQUE 2025 ALBUMS (proper counting): {true_album_count}")
print(f"   (Accounting for features, soundtracks, etc.)")

# Now recalculate accuracy with proper matching
new_predictions['primary_artist'] = new_predictions['Artist'].str.split(';').str[0].str.split(',').str[0].str.strip()
new_predictions['album_key'] = new_predictions['primary_artist'] + " | " + new_predictions['Album Name']

# Match with proper album keys
matches_clean = new_predictions[new_predictions['album_key'].isin(true_unique_albums['album_key'])]
true_accuracy_clean = len(matches_clean) / true_album_count * 100

print(f"\n🎯 TRUE ACCURACY (proper counting): {true_accuracy_clean:.1f}%")
print(f"   Correct predictions: {len(matches_clean)}/{true_album_count}")

# Show the actual unique album count
print(f"\n📊 SAMPLE OF UNIQUE ALBUMS:")
print(true_unique_albums[['Primary Artist', 'Album Name']].head(10).to_string(index=False))

🎵 TRUE UNIQUE 2025 ALBUMS (proper counting): 283
   (Accounting for features, soundtracks, etc.)

🎯 TRUE ACCURACY (proper counting): 89.4%
   Correct predictions: 253/283

📊 SAMPLE OF UNIQUE ALBUMS:
   Primary Artist                    Album Name
        The Hunts             Hibernating Heart
      Jim Nothing          Grey Eyes, Grey Lynn
  Franz Ferdinand                The Human Fear
Moonchild Sanelly                     Full Moon
       Mac Miller                  Balloonerism
            MEGGO eavesdropper ;; death stories
  Charlie Houston               Big After I Die
      Circa Waves           Death & Love, Pt. 1
  Heather Maloney                Exploding Star
Penny and Sparrow                         Lefty


In [27]:
# Analyze the prediction file
pred_file = './predictions/02-07-25_Album_Recommendations.csv'
df_preds = pd.read_csv(pred_file)

print("📊 PREDICTION FILE ANALYSIS")
print("="*50)
print(f"File: {pred_file}")
print(f"Shape: {df_preds.shape}")
print(f"Albums predicted: {len(df_preds)}")

print(f"\n📋 DATA DICTIONARY:")
print("="*50)
for col in df_preds.columns:
    dtype = df_preds[col].dtype
    sample = df_preds[col].iloc[0] if len(df_preds) > 0 else "N/A"
    print(f"• {col} ({dtype}): {sample}")

print(f"\n📈 SCORE DISTRIBUTION:")
print("="*50)
print(f"avg_score range: {df_preds['avg_score'].min():.1f} - {df_preds['avg_score'].max():.1f}")
print(f"confidence_score range: {df_preds['confidence_score'].min():.1f} - {df_preds['confidence_score'].max():.1f}")
print(f"Artist_Centrality range: {df_preds['Artist_Centrality'].min():.1f} - {df_preds['Artist_Centrality'].max():.1f}")
print(f"Mood_Score range: {df_preds['Mood_Score'].min():.1f} - {df_preds['Mood_Score'].max():.1f}")
print(f"Energy_Profile range: {df_preds['Energy_Profile'].min():.1f} - {df_preds['Energy_Profile'].max():.1f}")

print(f"\n🎵 TOP 10 PREDICTED ALBUMS:")
print("="*50)
top_10 = df_preds.nlargest(10, 'avg_score')[['Artist', 'Album Name', 'avg_score', 'Genres']]
for i, (_, album) in enumerate(top_10.iterrows(), 1):
    print(f"{i:2d}. {album['Artist']} - {album['Album Name']} ({album['avg_score']:.1f})")

# How to use as feature in end-of-year model
print(f"\n🚀 INTEGRATION WITH END-OF-YEAR MODEL:")
print("="*50)
print("""
USE CASES AS FEATURES:

1. EARLY_SIGNAL_STRENGTH:
   - Use avg_score as early prediction confidence
   - Albums with high scores in Feb are strong candidates

2. PREDICTION_TRAJECTORY:  
   - Track how predictions change over weekly runs
   - Albums that maintain high scores = reliable predictions

3. GENRE_ALIGNMENT:
   - Compare predicted genres with actual end-of-year preferences
   - See which genre predictions were most accurate

4. ARTIST_DISCOVERY_POWER:
   - Use Artist_Centrality to find new artists you ended up loving
   - High centrality + actual listening = successful discovery

CONCRETE INTEGRATION:

# Add to your training data as features:
df['feb_prediction_score'] = # Map from this file
df['feb_confidence'] = # Map confidence_score  
df['predicted_mood'] = # Map Mood_Score
df['predicted_energy'] = # Map Energy_Profile
df['early_artist_centrality'] = # Map Artist_Centrality

# Then train model to see if early predictions improve end-of-year accuracy!
""")

📊 PREDICTION FILE ANALYSIS
File: ./predictions/02-07-25_Album_Recommendations.csv
Shape: (253, 10)
Albums predicted: 253

📋 DATA DICTIONARY:
• Artist (object): Vincent Lima
• Album Name (object): To Love A Thing That Fades
• avg_score (float64): 100.0
• confidence_score (float64): 93.75
• track_count (int64): 15
• Genres (object): folk, pop
• Label (object): Blue Shutters Music LLC / Island Records
• Artist_Centrality (float64): 20.17
• Mood_Score (float64): 21.3
• Energy_Profile (float64): 68.41

📈 SCORE DISTRIBUTION:
avg_score range: 31.8 - 100.0
confidence_score range: 1.0 - 97.3
Artist_Centrality range: 1.0 - 66.9
Mood_Score range: 6.3 - 86.6
Energy_Profile range: 18.9 - 84.5

🎵 TOP 10 PREDICTED ALBUMS:
 1. Vincent Lima - To Love A Thing That Fades (100.0)
 2. Hand Habits - Blue Reminder (100.0)
 3. Runaway Brother - Want You Need (100.0)
 4. The Loft - Everything Changes Everything Stays The Same (100.0)
 5. SAINt JHN - FESTIVAL SEASON (100.0)
 6. Ellie Holcomb - Far Country (100.